In [1]:
import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
import hummingbird

    
torch.__version__


'1.5.0'

In [2]:
# We are going to use the breast cancer dataset from scikit-learn for this example.
X, y = load_breast_cancer(return_X_y=True)
nrows=15000
X = X[0:nrows].astype('|f4')
y = y[0:nrows]

In [4]:
# Create and train a random forest model.
model = RandomForestClassifier(n_estimators=10, max_depth=10)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [5]:
%%timeit -r 3

# Time for scikit-learn.
model.predict(X)

1.49 ms ± 16.9 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


In [12]:
hb_model = model.to_pytorch(extra_config={"tree_implementation":"gemm"})


In [13]:
%%timeit -r 3

# Time for HB.
hb_model.predict(X)

5.63 ms ± 34.9 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [13]:
hb_model.to('cuda')

PyTorchBackendModel(
  (operator_map): ModuleDict(
    (SklearnRandomForestClassifier): PerfectTreeTraversalDecisionTreeImpl(
      (nodes): ParameterList(
          (0): Parameter containing: [torch.cuda.LongTensor of size 20 (GPU 0)]
          (1): Parameter containing: [torch.cuda.LongTensor of size 40 (GPU 0)]
          (2): Parameter containing: [torch.cuda.LongTensor of size 80 (GPU 0)]
          (3): Parameter containing: [torch.cuda.LongTensor of size 160 (GPU 0)]
          (4): Parameter containing: [torch.cuda.LongTensor of size 320 (GPU 0)]
          (5): Parameter containing: [torch.cuda.LongTensor of size 640 (GPU 0)]
          (6): Parameter containing: [torch.cuda.LongTensor of size 1280 (GPU 0)]
          (7): Parameter containing: [torch.cuda.LongTensor of size 2560 (GPU 0)]
      )
      (biases): ParameterList(
          (0): Parameter containing: [torch.cuda.FloatTensor of size 20 (GPU 0)]
          (1): Parameter containing: [torch.cuda.FloatTensor of size 40 (GPU 

In [14]:
%%timeit -r 3

# Time for H GPUB.
hb_model.predict(X)

1.5 ms ± 13.1 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)
